In [69]:
#import dependencies
import gmaps
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)

In [70]:
path = '../weatherpy/outputs/cities.csv'
cities_for_heatmap_df = pd.read_csv(path)
cities_for_heatmap_df.drop(columns='Unnamed: 0', inplace=True)
cities_for_heatmap_df.sample(10)

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement
179,zyryanka,65.75,150.85,-17.09,100,87,11.34,RU,1608823796
110,neryungri,56.67,124.65,-17.25,100,86,4.05,RU,1608823792
104,oranjestad,12.52,-70.03,86.00,20,66,18.34,AW,1608823706
552,medicine hat,50.05,-110.67,35.60,90,55,13.87,CA,1608823847
479,chake chake,-5.25,39.77,81.27,25,77,11.05,TZ,1608823841
420,esperance,-33.87,121.90,68.00,0,64,16.11,AU,1608823836
238,lolodorf,3.23,10.73,79.68,28,81,3.80,CM,1608823799
152,haian,32.54,120.45,44.01,100,73,1.92,CN,1608823794
53,qandala,11.47,49.87,75.52,0,62,8.19,SO,1608823788
359,aras,42.56,-2.36,51.80,75,53,8.05,ES,1608823830


In [71]:
cities_for_heatmap_df['humidity'].max()

100

In [72]:
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=6)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [155]:
ideal_weather_df = pd.DataFrame(cities_for_heatmap_df[(cities_for_heatmap_df['max_temp'] < 70) & (cities_for_heatmap_df['max_temp'] > 60) & (cities_for_heatmap_df['wind_speed'] < 10) & (cities_for_heatmap_df['cloudiness'] == 0)])
ideal_weather_df['hotel_name'] = " "
ideal_weather_df

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,


In [162]:
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_keyword = 'hotel'
target_type = 'lodging'
for index, row in ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}, must drop row')
        hotel_names.append('n/a')
ideal_weather_df['hotel_name'] = hotel_names
ideal_weather_df

No hotel within 5000 m of tiznit, must drop row
No hotel within 5000 m of shadegan, must drop row


,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,Victoria Hotel - Bredasdorp
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,n/a
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,Man Thi Tar Hotel Minbu
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,Vandenberg Hotel
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,n/a
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,The First Hotel
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,WORACHAT BOUTIQUE HOTEL
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,فندق الزيتونة
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,Shapur Hotel


In [163]:
hotels_df = ideal_weather_df.drop(ideal_weather_df[ideal_weather_df.hotel_name == 'n/a'].index)
hotels_df

,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,Victoria Hotel - Bredasdorp
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,Man Thi Tar Hotel Minbu
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,Vandenberg Hotel
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,The First Hotel
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,WORACHAT BOUTIQUE HOTEL
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,فندق الزيتونة
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,Shapur Hotel


In [171]:
locations = cities_for_heatmap_df[['lat', 'long']]
weights = cities_for_heatmap_df['humidity']
central_coordinates = (0, 0)
fig = gmaps.figure(center=central_coordinates, zoom_level=1.5)
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights, max_intensity=100, point_radius=2, dissipating=False)
heatmap_layer.opacity = 0.5
fig.add_layer(heatmap_layer)
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{hotel_name}</dd>
<dt>City</dt><dd>{city_name}</dd>
<dt>Country</dt><dd>{country_code}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotels_df.iterrows()]
hotel_locations = hotels_df[["lat", "long"]]


marker_layer = gmaps.marker_layer(hotel_locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [138]:
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 5000
target_keyword = 'hotel'
target_type = 'lodging'
for index, row in ideal_weather_df.iterrows():
    try:
        target_location = f'{row[1]},{row[2]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'No hotel within 5000 m of {row[0]}, will try 10000 m range')
        try:
            target_radius = 10000
            target_location = f'{row[1]},{row[2]}'
            final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
            data = requests.get(final_url).json()
            hotel_names.append(f"{data['results'][0]['name']}* 10 km away")
            target_radius = 5000
        except IndexError:
            print(f'No hotel within 10000 m of {row[0]}, will try 20000 m range')
            try:
                target_radius = 20000
                target_location = f'{row[1]},{row[2]}'
                final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
                data = requests.get(final_url).json()
                hotel_names.append(f"{data['results'][0]['name']}* 20 km away")
                target_radius = 5000
            except IndexError:
                print(f'no hotel found within 20000 m of {row[0]}, exiting row')
                continue

ideal_weather_df['hotel_name'] = hotel_names
ideal_weather_df

No hotel within 5000 m of tiznit, will try 10000 m range
No hotel within 10000 m of tiznit, will try 20000 m range
No hotel within 5000 m of shadegan, will try 10000 m range
No hotel within 10000 m of shadegan, will try 20000 m range


,city_name,lat,long,max_temp,cloudiness,humidity,wind_speed,country_code,date_of_measurement,hotel_name
15,bredasdorp,-34.53,20.04,69.80,0,56,8.05,ZA,1608823786,Bredasdorp Square
102,tiznit,29.58,-9.50,66.16,0,27,6.51,MA,1608823791,Hotel kerdous* 20 km away
368,magway,20.15,94.92,67.59,0,62,3.40,MM,1608823831,Htain Htain Thar Hotel 1
426,forbes,-33.38,148.02,64.99,0,52,7.23,AU,1608823836,Appleyard Bed and Breakfast
480,shadegan,30.65,48.66,60.80,0,72,9.17,IR,1608823841,Estuary fishing pier Douragh* 20 km away
487,pathein,16.78,94.73,67.95,0,79,5.26,MM,1608823823,The First Hotel
493,lom sak,16.78,101.24,68.95,0,76,3.80,TH,1608823842,WORACHAT BOUTIQUE HOTEL
548,bani walid,31.76,13.99,62.22,0,50,5.14,LY,1608823847,فندق الزيتونة
551,turbat,26.00,63.04,63.84,0,30,4.43,PK,1608823847,Shapur Hotel


In [128]:
#testing to see if error is because of lack of hotels w/in 5000 m; increasing to 10000 m, then 20000 m
target_coords = [[29.58,-9.50], [30.65,48.66]]
hotel_names = []
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
key=g_key
target_radius = 20000
target_keyword = 'hotel'
target_type = 'lodging'
for item in target_coords:
    try:
        target_location = f'{item[0]},{item[1]}'
        final_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={target_location}&radius={target_radius}&keyword={target_keyword}&type={target_type}&key={api_key}'
        data = requests.get(final_url).json()
        hotel_names.append(data['results'][0]['name'])
    except IndexError:
        print(f'Error with {row[0]}')
        hotel_names.append('n/a')
        continue
hotel_names

['Hotel kerdous', 'Estuary fishing pier Douragh']